In [33]:
import subprocess
subprocess.run(["sudo","cpupower","frequency-set","-u","3.0GHz"],stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)

CompletedProcess(args=['sudo', 'cpupower', 'frequency-set', '-u', '3.0GHz'], returncode=0)

In [34]:
import pandas as pd
import torch.nn as nn
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [35]:
df=pd.read_csv("https://raw.githubusercontent.com/Arannamoy/datasets/refs/heads/main/Medical%20Cost%20Personal%20Datasets/insurance.csv")

In [36]:
df.head(5)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [37]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [39]:
categorical_cols=['sex','smoker','region']
label_encoder={}
train_df,test_df=train_test_split(df,test_size=0.25,random_state=42)
for col in categorical_cols:
    le=LabelEncoder()
    train_df[col]=le.fit_transform(train_df[col].astype(str))
    test_df[col]=le.transform(test_df[col].astype(str))
    label_encoder[col]=le

In [40]:
print(label_encoder['sex'].classes_)
print(label_encoder['region'].classes_)
print(label_encoder['smoker'].classes_)



for col,item in label_encoder.items():
    print(col,item.classes_)

    for index,label in enumerate(item.classes_):
        print(index,label)
        print(label_encoder[col].inverse_transform([index]))

['female' 'male']
['northeast' 'northwest' 'southeast' 'southwest']
['no' 'yes']
sex ['female' 'male']
0 female
['female']
1 male
['male']
smoker ['no' 'yes']
0 no
['no']
1 yes
['yes']
region ['northeast' 'northwest' 'southeast' 'southwest']
0 northeast
['northeast']
1 northwest
['northwest']
2 southeast
['southeast']
3 southwest
['southwest']


In [41]:
X_train=train_df.drop(columns=['charges'])
X_test=test_df.drop(columns=['charges'])
y_train=train_df['charges']
y_test=test_df['charges']

In [42]:
class SimpleNNLinearRegression(nn.Module):
    def __init__(self, input_layer,hidden_layer,output_layer):
        super(SimpleNNLinearRegression,self).__init__()
        self.network=nn.Sequential(
            
            nn.Linear(input_layer,hidden_layer),
            nn.ReLU(),
            nn.Linear(hidden_layer,output_layer),
            nn.ReLU(),
            nn.Linear(output_layer,1)
        
        )
    def forward(self,x):
        return self.network(x)

In [43]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
y_train=scaler.fit_transform(y_train.values.reshape(-1,1))
y_test=scaler.transform(y_test.values.reshape(-1,1))
X_train_tensor=torch.tensor(X_train,dtype=torch.float32)
X_test_tensor=torch.tensor(X_test,dtype=torch.float32)
y_test_tensor=torch.tensor(y_test,dtype=torch.float32) ### it based on nn.linear(output_layer,1), here is 1 final neuron so 1 column.
y_train_tensor=torch.tensor(y_train,dtype=torch.float32)

In [44]:
print(type(X_train_tensor.shape[1]))

<class 'int'>


In [45]:
model=SimpleNNLinearRegression(X_train_tensor.shape[1],64,128)

In [46]:
criterion=nn.MSELoss()
optimizer=optim.Adam(model.parameters(),lr=0.01)

In [47]:
epochs=100000
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    result=model(X_train_tensor)
    loss=criterion(result,y_train_tensor)
    loss.backward()
    optimizer.step()
    
    if (epoch+1)%10000==0:
        print(f"epoch {epoch}/{epochs}, Loss: {loss}")

epoch 9999/100000, Loss: 0.002212610561400652
epoch 19999/100000, Loss: 0.0015654698945581913
epoch 29999/100000, Loss: 0.0014023272087797523
epoch 39999/100000, Loss: 0.0012662598164752126
epoch 49999/100000, Loss: 0.0018421048298478127
epoch 59999/100000, Loss: 0.0012696179328486323
epoch 69999/100000, Loss: 0.0012602964416146278
epoch 79999/100000, Loss: 0.0013390149688348174
epoch 89999/100000, Loss: 0.0012603247305378318
epoch 99999/100000, Loss: 0.001263354206457734


In [48]:
subprocess.run(["sudo","cpupower","frequency-set","-u","2.8GHz"],stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)

CompletedProcess(args=['sudo', 'cpupower', 'frequency-set', '-u', '2.8GHz'], returncode=0)